In [2]:
import gradio as gr
import torch
import torchaudio
from transformers import AutoProcessor, AutoModelForSpeechSeq2Seq
import json
import datetime
from typing import Dict, List, Any
import numpy as np
from dataclasses import dataclass
import tempfile
import os

# =====================
# Configuration
# =====================
@dataclass
class FinanceBotConfig:
    """Configuration settings for the finance chatbot"""
    model_name: str = "ibm-granite/granite-speech-3.3-2b"
    device: str = "cuda" if torch.cuda.is_available() else "cpu"
    torch_dtype: torch.dtype = torch.bfloat16 if torch.cuda.is_available() else torch.float32
    sample_rate: int = 16000
    max_audio_length: int = 30  # seconds
    system_prompt: str = (
        "Knowledge Cutoff Date: April 2024.\n"
        "Today's Date: April 9, 2025.\n"
        "You are Granite Financial Assistant, developed by IBM. "
        "You are a helpful AI assistant specialized in personal finance, "
        "providing guidance on savings, investments, taxes, and budgeting."
    )

# =====================
# Granite Speech Processor
# =====================
class GraniteSpeechProcessor:
    """IBM Granite Speech-to-Text processor for voice input"""

    def __init__(self, config: FinanceBotConfig):
        self.config = config
        self.processor = None
        self.model = None
        self.tokenizer = None
        self.error = None
        self._initialize_model()

    def _initialize_model(self):
        """Initialize the Granite speech model"""
        try:
            self.processor = AutoProcessor.from_pretrained(self.config.model_name)
            self.tokenizer = self.processor.tokenizer
            # Use device_map="auto" for best compatibility
            self.model = AutoModelForSpeechSeq2Seq.from_pretrained(
                self.config.model_name,
                device_map="auto",
                torch_dtype=self.config.torch_dtype,
            )
        except Exception as e:
            self.error = f"Error loading Granite Speech Model: {str(e)}"
            # Fall back to text-only mode
            self.processor = None
            self.model = None
            self.tokenizer = None


    def transcribe_audio(self, file_path: str, user_prompt: str | None = None) -> str:
        """Transcribe audio using Granite Speech model.
        Accepts a filesystem path (from gr.Audio with type="filepath").
        """
        if not self.model or not self.processor or not self.tokenizer:
            return "Speech processing unavailable. Please use text input."
        try:
            wav, sr = torchaudio.load(file_path, normalize=True)

            # Ensure mono, 16kHz
            if wav.shape[0] > 1:
                wav = torch.mean(wav, dim=0, keepdim=True)
            if sr != self.config.sample_rate:
                resampler = torchaudio.transforms.Resample(sr, self.config.sample_rate)
                wav = resampler(wav)

            # Optional trim for very long files
            max_len = self.config.max_audio_length * self.config.sample_rate
            if wav.shape[-1] > max_len:
                wav = wav[..., :max_len]

            if not user_prompt:
                user_prompt = "<|audio|>Please transcribe this financial question or statement."

            chat = [
                dict(role="system", content=self.config.system_prompt),
                dict(role="user", content=user_prompt),
            ]

            prompt = self.tokenizer.apply_chat_template(
                chat, tokenize=False, add_generation_prompt=True
            )

            model_inputs = self.processor(
                prompt, wav, device=self.config.device, return_tensors="pt"
            )

            # Move tensors if needed
            if self.config.device == "cuda":
                model_inputs = {k: (v.to("cuda") if hasattr(v, "to") else v) for k, v in model_inputs.items()}

            model_outputs = self.model.generate(
                **model_inputs, max_new_tokens=200, do_sample=False, num_beams=1
            )

            num_input_tokens = model_inputs["input_ids"].shape[-1]
            new_tokens = torch.unsqueeze(model_outputs[0, num_input_tokens:], dim=0)
            output_text = self.tokenizer.batch_decode(
                new_tokens, add_special_tokens=False, skip_special_tokens=True
            )
            return output_text[0].strip()
        except Exception as e:
            return f"Error processing audio: {str(e)}"


# =====================
# Finance Brain
# =====================
class IBMFinanceAI:
    """Enhanced IBM Watson/Granite AI for comprehensive financial guidance"""

    def __init__(self):
        self.financial_knowledge = {
            'savings': {
                'emergency_fund': "Build 3-6 months of expenses for emergencies",
                'high_yield': "Consider high-yield savings accounts (4-5% APY)",
                'automation': "Automate savings with direct deposit splits",
                'goals': "Set specific savings goals with timelines"
            },
            'investments': {
                'diversification': "Diversify across stocks, bonds, and sectors",
                'index_funds': "Low-cost index funds (expense ratio < 0.1%)",
                'dollar_cost_averaging': "Invest consistently regardless of market timing",
                'retirement': "Maximize employer 401k match (free money)",
                'risk_tolerance': "Align investments with your risk profile"
            },
            'taxes': {
                'deductions': "Maximize pre-tax contributions (401k, IRA)",
                'tax_loss_harvesting': "Offset capital gains with losses",
                'brackets': "Understand marginal vs effective tax rates",
                'planning': "Consider Roth vs traditional accounts"
            },
            'budgeting': {
                '50_30_20': "50% needs, 30% wants, 20% savings/debt",
                'zero_based': "Assign every dollar a specific purpose",
                'tracking': "Use apps or spreadsheets to monitor spending",
                'debt': "Pay off high-interest debt first (avalanche method)"
            }
        }


    def generate_response(self, query: str, user_type: str, context: Dict) -> str:
        query_lower = query.lower()
        is_student = user_type == "student"
        if any(word in query_lower for word in ['budget', 'spending', 'expenses', 'money management']):
            return self._generate_budget_advice(is_student, context)
        elif any(word in query_lower for word in ['invest', 'stocks', 'portfolio', 'retirement', '401k']):
            return self._generate_investment_advice(is_student, context)
        elif any(word in query_lower for word in ['save', 'savings', 'emergency', 'fund']):
            return self._generate_savings_advice(is_student, context)
        elif any(word in query_lower for word in ['tax', 'deduction', 'ira', 'roth']):
            return self._generate_tax_advice(is_student, context)
        elif any(word in query_lower for word in ['debt', 'loan', 'credit card', 'payment']):
            return self._generate_debt_advice(is_student, context)
        else:
            return self._generate_general_advice(is_student)

    # ---- (same content as original helper methods) ----
    def _generate_budget_advice(self, is_student: bool, context: Dict) -> str:
        if is_student:
            return """📚 Student Budget Guide:

Hey! Let's get your money organized with some student-friendly tips:

🎯 The 50/30/20 Rule (Student Edition):
• 50% - Essentials (tuition, rent, groceries, textbooks)
• 30% - Fun stuff (dining out, entertainment, hobbies)
• 20% - Savings & debt payments (even $25/month helps!)

💡 Money-Saving Hacks:
• Student discounts everywhere (Spotify, Adobe, restaurants)
• Buy used textbooks or rent them
• Cook meals in bulk (saves $200+ monthly)
• Use campus resources (gym, library, events)
• Share streaming subscriptions with friends

📱 Easy Tracking:
Try apps like Mint, YNAB (free for students), or even a simple notebook. The key is seeing where your money actually goes!
Start small - even tracking for one week will open your eyes! 👀"""
        else:
            return """💼 Professional Budget Strategy:

📊 Advanced Budgeting Framework:

Zero-Based Budgeting:
• Assign every dollar before you earn it
• Income - Expenses - Savings = $0
• Review and adjust monthly

💰 Optimal Expense Ratios:
• Housing: 25-30% of gross income
• Transportation: 10-15%
• Food: 10-12%
• Savings/Investments: 20%+
• Insurance: 5-10%

🔧 Professional Tools:
• YNAB for comprehensive tracking
• Personal Capital for investment tracking
• Spreadsheet templates for detailed analysis

⚡ Optimization Strategies:
• Negotiate recurring expenses (insurance, phone, internet)
• Automate bill payments to avoid fees
• Use cashback credit cards responsibly
• Track and optimize your biggest expense categories

Focus on the 80/20 rule - optimize your largest expenses first for maximum impact."""


    def _generate_investment_advice(self, is_student: bool, context: Dict) -> str:
        if is_student:
            return """🎓 Student Investment Starter Pack:

🚀 Start Small, Think Big:
You don't need thousands to start investing! Here's your roadmap:

📱 Easy Starting Points:
• Robo-advisors: Betterment, Wealthfront ($0 minimum)
• Low-cost brokers: Fidelity, Schwab (commission-free trading)
• Target-date funds: Set it and forget it approach

💡 Student-Friendly Strategy:
• Start with $25-50/month (seriously, that's enough!)
• Choose broad market index funds (low fees, diversified)
• Take advantage of compound interest TIME advantage
• Don't try to pick individual stocks yet

🎯 Priority Order:
1. Build small emergency fund ($500-1000)
2. Get any employer 401k match (if working)
3. Start investing in index funds
4. Increase contributions as income grows

Remember: Time in the market beats timing the market! Starting at 20 vs 30 can mean hundreds of thousands more at retirement! 📈"""
        else:
            return """📈 Professional Investment Strategy:

🎯 Comprehensive Portfolio Approach:

Asset Allocation Framework:
• Age-based rule: (110 - your age)% in stocks
• Example at 30: 80% stocks, 20% bonds
• Rebalance quarterly or semi-annually

💼 Tax-Advantaged Priority:
1. Max employer 401k match (immediate 100% return)
2. Max Roth IRA ($6,500/year) if income eligible
3. Max 401k contribution ($22,500/year)
4. Consider backdoor Roth if high income
5. Taxable investments for additional goals

📊 Low-Cost Index Fund Portfolio:
• Total Stock Market Index: 60%
• International Stock Index: 20%
• Bond Index: 20%
• Target expense ratios: <0.1%

⚖ Advanced Strategies:
• Dollar-cost averaging into volatile markets
• Tax-loss harvesting in taxable accounts
• Consider target-date funds for simplicity
• Diversify across sectors and geographies

Focus on consistent contributions and low fees - these factors matter more than perfect timing."""


    def _generate_savings_advice(self, is_student: bool, context: Dict) -> str:
        if is_student:
            return """💰 Student Savings Made Simple:

🎯 Your Savings Game Plan:

Emergency Fund (Start Here!):
• Goal: $500-1,000 to start (covers most emergencies)
• Save $25-50/month until you reach it
• Keep it in a separate high-yield savings account
• Don't touch it unless it's a real emergency!

💡 Easy Saving Tricks:
• Round up purchases and save the change
• Save your tax refund instead of spending it
• Put half of any windfalls (gifts, work bonuses) into savings
• Try a "no-spend" challenge for a week each month

🏦 Where to Keep It:
• High-yield online savings (4-5% APY)
• Popular options: Ally, Marcus, Capital One 360
• Avoid big bank savings (they pay almost nothing!)

🎪 Make It Fun:
• Savings challenges (52-week challenge)
• Visualize your goals with charts
• Celebrate milestones ($100, $500, $1000)

Building the habit matters more than the amount right now! 🌟"""
        else:
            return """🏦 Professional Savings Strategy:

🎯 Tiered Emergency Fund Approach:

Emergency Fund Targets:
• Starter: 1 month of expenses
• Standard: 3-6 months of expenses
• Enhanced: 6-12 months (if self-employed/unstable income)

🔄 Systematic Savings Plan:
• Automate 20%+ of gross income to savings
• Direct deposit split: checking (80%) + savings (20%)
• Increase savings rate with every raise (lifestyle inflation protection)

💰 High-Yield Account Optimization:
• Target 4-5% APY (2024-2025 rates)
• FDIC insured up to $250,000
• No monthly fees or minimum balance requirements
• Consider CD ladders for higher rates on longer-term savings

📊 Goal-Based Savings Buckets:
• Emergency fund (3-6 months expenses)
• Short-term goals (<2 years): house down payment, vacation
• Medium-term goals (2-5 years): car, home improvements
• Long-term goals (5+ years): retirement, children's education

⚡ Advanced Strategies:
• I Bonds for inflation protection (up to $10k/year)
• Money market accounts for slightly higher rates
• Treasury bills for short-term, risk-free returns

Automate everything and let compound interest work for you while you sleep! 🚀"""

    def _generate_debt_advice(self, is_student: bool, context: Dict) -> str:
        if is_student:
            return """📚 Student Debt Management:

🎯 Smart Debt Strategy:

Student Loan Tips:
• Know your interest rates and loan types
• Consider income-driven repayment plans
• Pay extra toward highest interest rate loans first
• Don't ignore them - they don't disappear!

Credit Card Basics:
• Pay off FULL balance every month
• Keep utilization under 30% (ideally under 10%)
• Build credit history with responsible use
• Avoid cash advances and high fees

📊 Debt Payoff Priority:
1. Credit cards (highest interest first)
2. Private student loans
3. Federal student loans (often lowest rates)

💡 Extra Money Strategy:
• Tax refunds → debt payments
• Side hustle income → debt payments
• Skip one restaurant meal/week → extra payment

Remember: Every extra dollar toward debt saves you money in interest! 💪"""
        else:
            return """⚖ Professional Debt Management:

🎯 Strategic Debt Elimination:

Debt Avalanche Method (Mathematically Optimal):
• List all debts by interest rate (highest to lowest)
• Pay minimums on all debts
• Put all extra money toward highest rate debt
• Move to next highest rate when first is paid off

💳 Credit Optimization:
• Maintain credit utilization under 10%
• Pay balances before statement date
• Keep old accounts open (length of credit history)
• Monitor credit score monthly (Credit Karma, etc.)

🏠 Mortgage Strategies:
• Consider extra principal payments
• Refinance if rates drop 0.5-1% below current rate
• Weigh extra payments vs. investing (if mortgage rate < investment returns)

📊 Good Debt vs. Bad Debt:
• Good: Mortgage, student loans (tax deductible, low rates)
• Bad: Credit cards, personal loans, payday loans
• Focus elimination efforts on bad debt first

⚡ Advanced Tactics:
• 0% balance transfer offers (with discipline)
• Debt consolidation (if it lowers overall rate)
• Negotiate with creditors for better terms

Calculate the true cost of debt - a $5,000 credit card balance at 18% costs $900/year in interest alone! 📊"""

    def _generate_tax_advice(self, is_student: bool, context: Dict) -> str:
        if is_student:
            return """📋 Student Tax Basics:

🎓 Student-Specific Tax Benefits:

Education Credits:
• American Opportunity Credit: Up to $2,500/year
• Lifetime Learning Credit: Up to $2,000/year
• Requirements: Enrolled half-time+ in degree program

💰 Common Student Deductions:
• Student loan interest (up to $2,500)
• Tuition and fees (if eligible)
• Educational expenses for work/career

📱 Easy Filing:
• Use free tax software (TurboTax Free, FreeTaxUSA)
• Most students can file for free
• File even if you didn't earn much (you might get money back!)

💡 Pro Tips:
• Keep receipts for educational expenses
• Ask parents about claiming you as dependent (might be better for family)
• File your FAFSA early with completed taxes

Don't stress too much - student taxes are usually pretty straightforward! 😊"""
        else:
            return """📊 Professional Tax Strategy:

🎯 Tax Optimization Framework:

Pre-Tax Contribution Maximization:
• 401k: $22,500 limit (2023)
• Traditional IRA: $6,500 limit
• HSA: $3,850 individual/$7,750 family (triple tax advantage!)
• Reduce current tax burden while saving for future

💼 Professional Deductions:
• Home office (if working remotely)
• Professional development courses
• Work-related travel expenses
• Professional memberships and subscriptions

⚖ Roth vs. Traditional Decision:
• Traditional: Tax deduction now, taxed in retirement
• Roth: No deduction now, tax-free in retirement
• Consider: Current vs. future tax brackets
• Hedge with mix of both if uncertain

📈 Tax-Loss Harvesting:
• Realize losses to offset capital gains
• Be aware of wash sale rules (30-day rule)
• Rebalance portfolio while minimizing taxes

🔍 Advanced Strategies:
• Backdoor Roth IRA (if income too high for direct Roth)
• Mega backdoor Roth (if 401k allows after-tax contributions)
• Tax-efficient fund placement (bonds in tax-advantaged accounts)

Work with a CPA for complex situations - the tax code has many opportunities for optimization! 📚"""

    def _generate_general_advice(self, is_student: bool) -> str:
        if is_student:
            return """🌟 General Financial Wisdom for Students:

🎯 Your Financial Foundation:

The Big Picture:
Building wealth is like planting a tree - the best time was 20 years ago, the second best time is now! You have the most valuable asset: TIME.

💡 Key Principles:
• Pay yourself first (save before spending)
• Live below your means (lifestyle inflation is real)
• Invest in yourself (education, skills, health)
• Start investing ASAP (compound interest is magic)
• Build good credit early

🎪 Make Money Fun:
• Set small, achievable goals
• Celebrate milestones
• Use apps and tools that gamify saving
• Learn from others (podcasts, books, friends)

📚 Resources to Check Out:
• Books: "The Total Money Makeover," "The Simple Path to Wealth"
• Podcasts: "The Dave Ramsey Show," "ChooseFI"
• YouTube: Ben Felix, Two Cents

You're already ahead of the game by asking these questions! 🚀"""
        else:
            return """💼 Comprehensive Financial Strategy:

🎯 Holistic Wealth Building:

The Professional's Financial Framework:
1. Foundation: Emergency fund + insurance protection
2. Growth: Maximize tax-advantaged investments
3. Optimization: Tax strategies + estate planning
4. Acceleration: Alternative investments + business opportunities

⚖ Key Wealth Principles:
• Increase income aggressively (skills, negotiations, side businesses)
• Optimize tax efficiency across all accounts
• Diversify across asset classes and geographies
• Regularly rebalance and review strategy
• Plan for major life changes (marriage, children, career changes)

📊 Quarterly Financial Reviews:
• Net worth tracking and goal assessment
• Investment performance and rebalancing
• Tax planning and strategy adjustments
• Insurance needs review
• Estate planning updates

🚀 Advanced Wealth Strategies:
• Consider real estate investment (REITs or direct)
• Explore tax-advantaged business structures
• Investigate international diversification
• Plan charitable giving for tax benefits
• Build multiple income streams

The key is systematic approach with regular optimization - wealth building is a marathon, not a sprint! 📈"""

    # ---- Budget summary helpers ----
    def generate_budget_summary(self, expenses: Dict, income: float, user_type: str) -> str:
        total_expenses = sum(expenses.values())
        savings_rate = ((income - total_expenses) / income) * 100 if income > 0 else 0
        analysis = f"""
🤖 AI Budget Analysis:

📊 Financial Health Score: {self._calculate_health_score(income, expenses)}/100

💰 Cash Flow:
• Monthly Income: ${income:,.2f}
• Total Expenses: ${total_expenses:,.2f}
• Net Savings: ${income - total_expenses:,.2f}
• Savings Rate: {savings_rate:.1f}%

🎯 Recommendations:
{self._generate_budget_recommendations(income, expenses, user_type)}

📈 Action Items:
{self._generate_action_items(income, expenses, user_type)}
"""
        return analysis

    def _calculate_health_score(self, income: float, expenses: Dict) -> int:
        if income <= 0:
            return 0
        total_expenses = sum(expenses.values())
        savings_rate = ((income - total_expenses) / income) * 100
        score = 0
        if savings_rate >= 20:
            score += 40
        elif savings_rate >= 10:
            score += 25
        elif savings_rate >= 0:
            score += 15
        housing_ratio = (expenses.get('Housing', 0) / income) * 100
        if housing_ratio <= 30:
            score += 20
        elif housing_ratio <= 40:
            score += 10
        debt_payments = expenses.get('Debt Payments', 0)
        debt_ratio = (debt_payments / income) * 100
        if debt_ratio <= 10:
            score += 20
        elif debt_ratio <= 20:
            score += 15
        elif debt_ratio <= 30:
            score += 10
        score += 15  # assume moderate emergency fund
        return min(score, 100)

    def _generate_budget_recommendations(self, income: float, expenses: Dict, user_type: str) -> str:
        total_expenses = sum(expenses.values())
        savings_rate = ((income - total_expenses) / income) * 100 if income else 0
        recommendations = []
        if savings_rate < 10:
            recommendations.append("🚨 Increase savings rate - aim for at least 10-15%")
        elif savings_rate < 20:
            recommendations.append("📈 Good progress! Try to reach 20% savings rate")
        else:
            recommendations.append("🌟 Excellent savings rate! Consider investing excess")
        housing_ratio = (expenses.get('Housing', 0) / income) * 100 if income else 0
        if housing_ratio > 30:
            recommendations.append(f"🏠 Housing costs high ({housing_ratio:.0f}%) - consider downsizing")
        transport_ratio = (expenses.get('Transportation', 0) / income) * 100 if income else 0
        if transport_ratio > 15:
            recommendations.append(f"🚗 Transportation costs high ({transport_ratio:.0f}%) - review options")
        food_ratio = (expenses.get('Food', 0) / income) * 100 if income else 0
        if food_ratio > 12:
            recommendations.append(f"🍽 Food spending high ({food_ratio:.0f}%) - try meal planning")
        return "\n".join([f"• {rec}" for rec in recommendations])

    def _generate_action_items(self, income: float, expenses: Dict, user_type: str) -> str:
        actions = []
        if user_type == "student":
            actions.extend([
                "📱 Download a budgeting app (Mint, YNAB, or PocketGuard)",
                "🎓 Look for additional student discounts on current expenses",
                "💰 Set up automatic transfer to savings account ($25-50/month)",
            ])
        else:
            actions.extend([
                "📊 Set up automated bill pay to avoid late fees",
                "🏦 Review and negotiate recurring expenses (insurance, phone, etc.)",
                "💼 Increase 401k contribution if not maximizing employer match",
            ])
        if expenses.get('Entertainment', 0) > income * 0.1:
            actions.append("🎭 Review entertainment spending for optimization opportunities")
        if expenses.get('Shopping', 0) > income * 0.05:
            actions.append("🛒 Implement 24-hour rule before non-essential purchases")
        return "\n".join([f"• {a}" for a in actions[:5]])


# =====================
# Gradio App
# =====================

def build_interface():
    cfg = FinanceBotConfig()
    speech = GraniteSpeechProcessor(cfg)
    brain = IBMFinanceAI()


    with gr.Blocks(title="Personal Finance AI Assistant (Gradio)", theme=gr.themes.Soft()) as demo:
        gr.Markdown("# 🤖 Personal Finance AI Assistant\n*Powered by IBM Granite AI & Watson Technologies*")

        # Global state
        chat_state = gr.State([])  # list of (user, assistant)

        with gr.Row():
            with gr.Column(scale=1):
                gr.Markdown("## 👤 User Profile")
                user_type = gr.Dropdown(["student", "professional"], value="student", label="I am a:")
                monthly_income = gr.Number(value=3000.0, precision=2, label="Monthly Income ($)")
                age = gr.Slider(18, 70, value=25, step=1, label="Age")
                risk_tolerance = gr.Radio(["Conservative", "Moderate", "Aggressive"], value="Moderate", label="Investment Risk Tolerance")


                gr.Markdown("## 📊 Monthly Budget")
                with gr.Row():
                    Housing = gr.Number(value=500.0, label="Housing ($)")
                    Food = gr.Number(value=200.0, label="Food ($)")
                with gr.Row():
                    Transportation = gr.Number(value=200.0, label="Transportation ($)")
                    Entertainment = gr.Number(value=200.0, label="Entertainment ($)")
                with gr.Row():
                    Shopping = gr.Number(value=200.0, label="Shopping ($)")
                    Healthcare = gr.Number(value=200.0, label="Healthcare ($)")
                with gr.Row():
                    Debt_Payments = gr.Number(value=200.0, label="Debt Payments ($)")
                    Other = gr.Number(value=200.0, label="Other ($)")


                generate_summary_btn = gr.Button("📈 Generate Budget Summary", variant="secondary")
                budget_summary = gr.Markdown("")


            with gr.Column(scale=2):
                gr.Markdown("## 💬 Ask Your Finance Questions")

                with gr.Tabs():
                    with gr.Tab("Text"):
                        user_query = gr.Textbox(lines=4, placeholder="e.g., How should I start investing? What's the best budgeting method for students?", label="Your question")
                        ask_btn = gr.Button("Get Financial Advice", variant="primary")
                    with gr.Tab("Voice (Speech-to-Text)"):
                        gr.Markdown("Upload or record audio, then click Transcribe to fill the text box.")
                        audio = gr.Audio(sources=["microphone", "upload"], type="filepath")
                        transcribe_btn = gr.Button("🎤 Transcribe with Granite")
                        transcribed_text = gr.Textbox(label="Transcribed Text", interactive=False)
                        fill_to_query_btn = gr.Button("➡ Use as Question")


                chat = gr.Chatbot(label="Chat History", height=400)
                clear_btn = gr.Button("🧹 Clear Chat")

        # =====================
        # Functions
        # =====================
        def on_transcribe(audio_path: str) -> str:
            if not audio_path:
                return "No audio provided."
            if speech.error:
                return f"{speech.error} — running in text-only mode."
            return speech.transcribe_audio(audio_path, "<|audio|>Please transcribe this personal finance question.")

        def on_fill_query(transcribed: str):
            return gr.update(value=transcribed)

        def on_ask(question: str, cur_state: List, user_type_v: str, income_v: float,
                   age_v: int, risk_v: str,
                   Housing_v: float, Food_v: float, Transportation_v: float, Entertainment_v: float,
                   Shopping_v: float, Healthcare_v: float, Debt_Payments_v: float, Other_v: float):
            if not question or not str(question).strip():
                return cur_state, gr.update(value="Please enter a question." )
            expenses = {
                "Housing": Housing_v or 0.0,
                "Food": Food_v or 0.0,
                "Transportation": Transportation_v or 0.0,
                "Entertainment": Entertainment_v or 0.0,
                "Shopping": Shopping_v or 0.0,
                "Healthcare": Healthcare_v or 0.0,
                "Debt Payments": Debt_Payments_v or 0.0,
                "Other": Other_v or 0.0,
            }
            ctx = {
                'income': float(income_v or 0.0),
                'expenses': expenses,
                'age': int(age_v or 0),
                'risk_tolerance': risk_v,
            }
            answer = brain.generate_response(question, user_type_v, ctx)
            timestamp = datetime.datetime.now().strftime('%Y-%m-%d %H:%M')
            cur_state = (cur_state or []) + [
                (f"{question}", f"{answer}\n\n📅 {timestamp}")
            ]
            return cur_state, gr.update(value="")


        def on_clear():
            return [],


        def on_budget_summary(user_type_v: str, income_v: float,
                              Housing_v: float, Food_v: float, Transportation_v: float, Entertainment_v: float,
                              Shopping_v: float, Healthcare_v: float, Debt_Payments_v: float, Other_v: float):
            expenses = {
                "Housing": Housing_v or 0.0,
                "Food": Food_v or 0.0,
                "Transportation": Transportation_v or 0.0,
                "Entertainment": Entertainment_v or 0.0,
                "Shopping": Shopping_v or 0.0,
                "Healthcare": Healthcare_v or 0.0,
                "Debt Payments": Debt_Payments_v or 0.0,
                "Other": Other_v or 0.0,
            }
            md = brain.generate_budget_summary(expenses, float(income_v or 0.0), user_type_v)
            return md

        # =====================
        # Wiring
        # =====================
        transcribe_btn.click(on_transcribe, inputs=[audio], outputs=[transcribed_text])
        fill_to_query_btn.click(on_fill_query, inputs=[transcribed_text], outputs=[user_query])

        ask_btn.click(
            on_ask,
            inputs=[user_query, chat_state, user_type, monthly_income, age, risk_tolerance,
                    Housing, Food, Transportation, Entertainment, Shopping, Healthcare, Debt_Payments, Other],
            outputs=[chat, user_query],
        )
        clear_btn.click(on_clear, inputs=None, outputs=[chat])


        generate_summary_btn.click(
            on_budget_summary,
            inputs=[user_type, monthly_income, Housing, Food, Transportation, Entertainment,
                    Shopping, Healthcare, Debt_Payments, Other],
            outputs=[budget_summary]
        )

    return demo


if __name__ == "__main__":
    demo = build_interface()
    demo.launch()

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

config.json: 0.00B [00:00, ?B/s]

...

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/183 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/68.2M [00:00<?, ?B/s]

/usr/local/lib/python3.12/dist-packages/torch/nn/modules/module.py:2441: UserWarning: for language_model.model.layers.37.self_attn.q_proj.lora_A.default.weight: copying from a non-meta parameter in the checkpoint to a meta parameter in the current model, which is a no-op. (Did you mean to pass `assign=True` to assign items in the state dictionary to their corresponding key in the module instead of copying them in place?)
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torch/nn/modules/module.py:2441: UserWarning: for language_model.model.layers.37.self_attn.q_proj.lora_B.default.weight: copying from a non-meta parameter in the checkpoint to a meta parameter in the current model, which is a no-op. (Did you mean to pass `assign=True` to assign items in the state dictionary to their corresponding key in the module instead of copying them in place?)
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torch/nn/modules/module.py:2441: UserWarning: for language_model.model.layer

It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://a448ffa82800602071.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
